## Load all the Python Libraries Required

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input
from keras.applications.resnet50 import ResNet50
import keras.backend as K
from tensorflow.python.framework import graph_util, graph_io
import coremltools

Using TensorFlow backend.


### Set the learning phase using the backend as False and load the model

In [2]:
K.set_learning_phase(False)

# Load the model with the weights trained on Imagenet 
model = ResNet50(weights='imagenet',include_top=True)

### Convert the model into a coreml model

In [3]:
# Convert the loaded Keras model into a CoreML Model
# Provide image_input_names to specify it is an image model
# also provide the bias values specified by the model
mlmodel = coremltools.converters.keras.convert(model, 
                                               input_names=['image'], 
                                               image_input_names=['image'],
                                               output_names=['classLabelProbs', 'classLabel'],
                                               class_labels='labels.txt', 
                                               blue_bias=-123.68,
                                               green_bias=-116.779,
                                               red_bias=-103.939)

Output name length mismatch
0 : input_1, <keras.engine.topology.InputLayer object at 0xb2b5568d0>
1 : conv1_pad, <keras.layers.convolutional.ZeroPadding2D object at 0xd2babccc0>
2 : conv1, <keras.layers.convolutional.Conv2D object at 0xd2babca90>
3 : bn_conv1, <keras.layers.normalization.BatchNormalization object at 0xd2babceb8>
4 : activation_1, <keras.layers.core.Activation object at 0xd2babcf28>
5 : max_pooling2d_1, <keras.layers.pooling.MaxPooling2D object at 0x10bf78cf8>
6 : res2a_branch2a, <keras.layers.convolutional.Conv2D object at 0xd2bae4630>
7 : bn2a_branch2a, <keras.layers.normalization.BatchNormalization object at 0xd2bb78828>
8 : activation_2, <keras.layers.core.Activation object at 0xd2bc0c588>
9 : res2a_branch2b, <keras.layers.convolutional.Conv2D object at 0xd2bc38358>
10 : bn2a_branch2b, <keras.layers.normalization.BatchNormalization object at 0xd2bc38400>
11 : activation_3, <keras.layers.core.Activation object at 0xd2bc78f98>
12 : res2a_branch2c, <keras.layers.convol

106 : bn4c_branch2b, <keras.layers.normalization.BatchNormalization object at 0xd2dcc1940>
107 : activation_30, <keras.layers.core.Activation object at 0xd2dd38588>
108 : res4c_branch2c, <keras.layers.convolutional.Conv2D object at 0xd2ddcf828>
109 : bn4c_branch2c, <keras.layers.normalization.BatchNormalization object at 0xd2de03208>
110 : add_10, <keras.layers.merge.Add object at 0xd2de3ef98>
111 : activation_31, <keras.layers.core.Activation object at 0xd2df0c748>
112 : res4d_branch2a, <keras.layers.convolutional.Conv2D object at 0xd2df0c7b8>
113 : bn4d_branch2a, <keras.layers.normalization.BatchNormalization object at 0xd2df49eb8>
114 : activation_32, <keras.layers.core.Activation object at 0xd2df77a20>
115 : res4d_branch2b, <keras.layers.convolutional.Conv2D object at 0xd2e00a668>
116 : bn4d_branch2b, <keras.layers.normalization.BatchNormalization object at 0xd2e038160>
117 : activation_33, <keras.layers.core.Activation object at 0xd2e10a9e8>
118 : res4d_branch2c, <keras.layers.con

### Save the model as a .mlmodel file which can be deployed

In [4]:
mlmodel.save('ModelZoo/ResNet50.mlmodel')

### Now extract the model using the TensorFlow session to convert it for Android

In [5]:
S = K.get_session()
constant_graph = graph_util.convert_variables_to_constants(S, 
                                                           S.graph.as_graph_def(),
                                                           ['fc1000/Softmax'])
graph_io.write_graph(constant_graph, 'ModelZoo/', 'ResNet50.pb', as_text=False)

INFO:tensorflow:Froze 320 variables.


INFO:tensorflow:Froze 320 variables.


INFO:tensorflow:Converted 320 variables to const ops.


INFO:tensorflow:Converted 320 variables to const ops.


'ModelZoo/ResNet50.pb'

### To convert the model for TensorFlow Lite

In [6]:
# To convert the model for tensorflow lite, we can save the keras model
# and then convert it into a TensorFlow lite model directly

input_tensor = tf.keras.layers.Input(shape=(224,224,3))
model = tf.keras.applications.resnet50.ResNet50(
    weights='imagenet', input_tensor=input_tensor, include_top=True
)

keras_file = 'ModelZoo/ResNet50.h5'
tf.keras.models.save_model(model, keras_file)

converter = tf.contrib.lite.TFLiteConverter.from_keras_model_file(keras_file)
tflite_model = converter.convert()
open('ModelZoo/'+'ResNet50.tflite', 'wb').write(tflite_model)

INFO:tensorflow:Froze 320 variables.


INFO:tensorflow:Froze 320 variables.


INFO:tensorflow:Converted 320 variables to const ops.


INFO:tensorflow:Converted 320 variables to const ops.


102144208

### Now load the model and get the details of the converted model
### These are useful when deploying the models on Android

In [7]:
interpreter = tf.contrib.lite.Interpreter(model_path='ModelZoo/ResNet50.tflite')

interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details, output_details)

[{'name': 'input_1', 'index': 79, 'shape': array([  1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}] [{'name': 'fc1000/Softmax', 'index': 77, 'shape': array([   1, 1000], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
